In [28]:
# see https://github.com/pydantic/pydantic/discussions/9509

# this doesn't work for us because using __init__ functions that handle positional rather
# than keyword construction, as we do, interferes with Pydantic's type checking. Pending
# # asking the Pydantic folks for help, we kludge by naming the the field with the alias
#
# ...and when we can, we make users go to keyword rather than positional arguments.

from pydantic import BaseModel, ConfigDict, Field, ValidationError

# json_data = '{"lens_encoders" : 42}'
# json_data_alias = '{"lensEncoders" : 42}'
json_data = {"lens_encoders" : 42}
json_data_alias = {"lensEncoders" : 42}

class Model(BaseModel):

    model_config = ConfigDict(validate_assignment=True,
                              use_enum_values=True,
                              populate_by_name=True,
                              extra="forbid")

    lens_encoders: int = Field(alias="lensEncoders")

m0 = Model(lens_encoders=42)
print(f"m0.model_dump() -> {m0.model_dump()}")
assert(json_data == m0.model_dump())
print(f"m0.model_dump(by_alias=True) -> {m0.model_dump(by_alias=True)}")
assert(json_data_alias == m0.model_dump(by_alias=True))

try:
    m1 = Model.model_validate(json_data)
    print(f"validated {json_data} against model")
except ValidationError as e:
    print(f"failed to validate {json_data} against model: {e}")

try:
    m2 = Model.model_validate(json_data_alias)
    print(f"validated {json_data_alias} against model")
except ValidationError as e:
    print(f"failed to validate {json_data_alias} against model: {e}")

try:
    m0.lens_encoders = 65
    print("assigned to m0.lens_encoders OK")
except ValidationError as e:
    print(f"failed to assign to m0.lens_encoders: {e}")

# try:
#     m0.lensEncoders = 65
#     print("assigned to m0.lensEncoders OK")
# except ValidationError as e:
#     print(f"failed to assign to m0.lens_encoders: {e}")


m0.model_dump() -> {'lens_encoders': 42}
m0.model_dump(by_alias=True) -> {'lensEncoders': 42}
validated {'lens_encoders': 42} against model
validated {'lensEncoders': 42} against model
assigned to m0.lens_encoders OK
